In [35]:
import pandas as pd

train = pd.read_csv('../data/train/train.csv')
venue_item = pd.read_csv('../data/train/venue_item.csv')

In [36]:
# 주요 하이퍼파라미터
LOOKBACK, PREDICT = 28, 7 # 고정 (대회 규칙)
seed = 42

# 디바이스 설정
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Preprocess

In [37]:
def seasonal_zero_handling(train_df):
    """
    계절별로 매출이 있는 기간만 학습에 사용
    메뉴별로 첫 매출 ~ 마지막 매출 사이 기간만 필터링
    """
    processed_groups = []
    
    for store_menu, group in train_df.groupby('영업장명_메뉴명'):
        group = group.sort_values('영업일자').copy()
        
        # 첫 매출과 마지막 매출 사이만 사용
        first_sale = group[group['매출수량'] > 0]['영업일자'].min()
        last_sale = group[group['매출수량'] > 0]['영업일자'].max()
        
        if pd.notna(first_sale) and pd.notna(last_sale):
            mask = (group['영업일자'] >= first_sale) & (group['영업일자'] <= last_sale)
            processed_groups.append(group[mask])
        elif (group['매출수량'] > 0).any():  # 매출이 하루만 있는 경우
            processed_groups.append(group[group['매출수량'] > 0])
    
    return pd.concat(processed_groups, ignore_index=True) if processed_groups else train_df


In [38]:
# 음수 처리
train['매출수량'] = train['매출수량'].clip(lower=0)

In [39]:
# 업장, 메뉴 분리
sp = train["영업장명_메뉴명"].str.split("_", n=1, expand=True)
train["venue"] = sp[0]; train["item"] = sp[1]

In [40]:
# 날짜 형식 변환
train["영업일자"] = pd.to_datetime(train["영업일자"])

# 평일(월화수목)
train["is_weekday"] = train["영업일자"].dt.weekday < 4

# 고정일 공휴일(음력/대체공휴일 제외)
_fixed_mmdd = {"01-01","03-01","05-05","06-06","08-15","10-03","10-09","12-25"}
train["is_fixed_holiday"] = train["영업일자"].dt.strftime("%m-%d").isin(_fixed_mmdd)

In [41]:
# 카테고리, 특성 추가
train = train.merge(venue_item, left_on=['venue', 'item'], 
                   right_on=['venue', 'item'], how='left')

# 계절 특성
train['month'] = train['영업일자'].dt.month
train['season'] = train['month'].map({12:0, 1:0, 2:0,  # 겨울
                                     3:1, 4:1, 5:1,   # 봄  
                                     6:2, 7:2, 8:2,   # 여름
                                     9:3, 10:3, 11:3}) # 가을

In [42]:
train

,영업일자,영업장명_메뉴명,매출수량,venue,item,is_weekday,is_fixed_holiday,category,is_group,is_room_service,is_addon,is_disposable_or_condiment,is_zero_sugar,drink_temp_hot,drink_temp_ice,spicy_flag,kids_flag,month,season
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,느티나무 셀프BBQ,1인 수저세트,False,True,패키지·단체·대관·서비스·옵션,True,False,False,True,False,False,False,False,False,1,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,느티나무 셀프BBQ,1인 수저세트,True,False,패키지·단체·대관·서비스·옵션,True,False,False,True,False,False,False,False,False,1,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,느티나무 셀프BBQ,1인 수저세트,True,False,패키지·단체·대관·서비스·옵션,True,False,False,True,False,False,False,False,False,1,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,느티나무 셀프BBQ,1인 수저세트,True,False,패키지·단체·대관·서비스·옵션,True,False,False,True,False,False,False,False,False,1,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,느티나무 셀프BBQ,1인 수저세트,True,False,패키지·단체·대관·서비스·옵션,True,False,False,True,False,False,False,False,False,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,화담숲카페,현미뻥스크림,True,False,디저트·베이커리,False,False,False,False,False,False,False,False,False,6,2
102672,2024-06-12,화담숲카페_현미뻥스크림,10,화담숲카페,현미뻥스크림,True,False,디저트·베이커리,False,False,False,False,False,False,False,False,False,6,2
102673,2024-06-13,화담숲카페_현미뻥스크림,14,화담숲카페,현미뻥스크림,True,False,디저트·베이커리,False,False,False,False,False,False,False,False,False,6,2
102674,2024-06-14,화담숲카페_현미뻥스크림,12,화담숲카페,현미뻥스크림,False,False,디저트·베이커리,False,False,False,False,False,False,False,False,False,6,2


In [43]:
# # Zero Sales 처리 적용
# print("Zero Sales 처리 전:")
# print(f"전체 데이터: {len(train):,}개")
# print(f"0인 매출수량: {(train['매출수량'] == 0).sum():,}개 ({(train['매출수량']==0).mean():.3f})")

# train_filtered = seasonal_zero_handling(train)

# print("\nZero Sales 처리 후:")
# print(f"필터링된 데이터: {len(train_filtered):,}개")
# print(f"0인 매출수량: {(train_filtered['매출수량'] == 0).sum():,}개 ({(train_filtered['매출수량']==0).mean():.3f})")

# # 처리된 데이터로 업데이트
# train = train_filtered.copy()


## LightGBM

In [58]:
import os, re, glob
import numpy as np
import pandas as pd
from datetime import timedelta
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
from pandas.api.types import CategoricalDtype

# ====== 경로 설정 ======
# 프로젝트 구조에 맞게 수정하세요.
DATA_DIR   = "../data"              # train.csv, sample_submission.csv 등
TEST_DIR   = os.path.join(DATA_DIR, "test")   # TEST_00.csv ~ TEST_09.csv
MODEL_DIR  = "../model"
OUT_DIR    = "../output"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# ====== 컬럼/룰 설정 ======
DATE_COL   = "영업일자"
TARGET     = "매출수량"
KEY_COL    = "영업장명_메뉴명"
BOOL_COLS  = [
    "is_weekday","is_fixed_holiday","is_group","is_room_service",
    "is_addon","is_disposable_or_condiment","is_zero_sugar",
    "drink_temp_hot","drink_temp_ice","spicy_flag","kids_flag"
]
CAT_COLS   = ["venue","item","category","season"]  # 존재하는 것만 사용
FIXED_SOLAR_HOLIDAYS = {(1,1), (3,1), (5,5), (6,6), (8,15), (10,3), (10,9), (12,25)}  # 단순 고정 공휴일만

# ====== 학습/예측 하이퍼파라미터 ======
VAL_DAYS         = 35          # 마지막 35일 검증
LGBM_OBJECTIVE   = "poisson"   # 카운트면 poisson 권장 (필요시 "regression")
LGBM_PARAMS = dict(
    objective=LGBM_OBJECTIVE,
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=127,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

# 예측 후처리 옵션
CLIP_ZERO   = True   # 음수 방지
ROUND_PRED  = False  # 일단 소수 유지(제출 직전 캐스팅에서 정수 변환 권장)

def ensure_parent_dir(path: str):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)


In [45]:
def ensure_venue_item(df: pd.DataFrame) -> pd.DataFrame:
    """venue/item 컬럼이 없으면 '영업장명_메뉴명'을 분리해서 생성"""
    if "venue" not in df.columns or "item" not in df.columns:
        if KEY_COL in df.columns:
            sp = df[KEY_COL].astype(str).str.split("_", n=1, expand=True)
            df["venue"] = sp[0]
            df["item"]  = sp[1]
        else:
            raise ValueError("venue/item 컬럼이 없고, '영업장명_메뉴명'도 없어 분리가 불가합니다.")
    return df

def add_calendar_feats(df: pd.DataFrame) -> pd.DataFrame:
    """달력 파생 (dow, month, season, is_weekday, is_weekend, is_fixed_holiday)"""
    df["date"]  = pd.to_datetime(df[DATE_COL])
    df["dow"]   = df["date"].dt.weekday          # 0=월 ... 6=일
    df["month"] = df["date"].dt.month
    df["season"] = df["month"].map(
        lambda m: 0 if m in (12,1,2) else (1 if m in (3,4,5) else (2 if m in (6,7,8) else 3))
    )
    df["is_weekday"] = (df["dow"] < 5).astype("int8")
    df["is_weekend"] = (df["dow"] >= 5).astype("int8")
    if "is_fixed_holiday" not in df.columns:
        df["is_fixed_holiday"] = df["date"].map(lambda d: (d.month, d.day) in FIXED_SOLAR_HOLIDAYS).astype("int8")
    return df

def cast_dtypes(df: pd.DataFrame, fit_categories: dict | None = None):
    """LightGBM 친화적 dtype 정리 및 카테고리 정합성 유지"""
    # 불리언류 -> int8
    for c in BOOL_COLS + ["is_weekend"]:
        if c in df.columns:
            df[c] = df[c].astype("int8")

    # 카테고리 처리
    for c in CAT_COLS:
        if c in df.columns:
            df[c] = df[c].astype("category")
            if fit_categories is not None and c in fit_categories:
                df[c] = df[c].cat.set_categories(fit_categories[c])
    return df

def add_lag_roll_feats(df: pd.DataFrame, group_cols=["venue","item"], max_lag=28, roll_windows=[7,14,28]):
    """(venue,item)별 lag 1~max_lag & rolling mean/std (shifted)"""
    df = df.sort_values(group_cols + ["date"]).copy()
    # lag 1..28
    for lag in range(1, max_lag+1):
        df[f"lag_{lag}"] = df.groupby(group_cols)[TARGET].shift(lag)

    # rolling mean/std (당일 누출 방지 위해 shift 후 rolling)
    for win in roll_windows:
        rolled = (
            df.groupby(group_cols)[TARGET]
              .shift(1)  # 당일 제외
              .rolling(window=win, min_periods=2)
        )
        df[f"roll_mean_{win}"] = rolled.mean()
        df[f"roll_std_{win}"]  = rolled.std()
    return df

def get_feature_cols(df: pd.DataFrame):
    drop_cols = [DATE_COL, "date", KEY_COL, TARGET]
    return [c for c in df.columns if c not in drop_cols]

def smape(y_true, y_pred):
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff  = np.abs(y_true - y_pred) / np.where(denom==0, 1, denom)
    return 100 * np.mean(diff)


In [54]:
def train_lgbm_28to7(train_df: pd.DataFrame, val_days: int = VAL_DAYS, objective: str = LGBM_OBJECTIVE):
    """
    train_df: 최소 컬럼 [영업일자, 매출수량, venue, item] (+ 있으면 좋은 컬럼들)
    val_days: 마지막 val_days일을 검증 기간으로 사용 (시간 홀드아웃)
    objective: "poisson" (카운트) 또는 "regression"
    """
    df = train_df.copy()
    df = ensure_venue_item(df)
    df = add_calendar_feats(df)
    df = add_lag_roll_feats(df, max_lag=28, roll_windows=[7,14,28])

    # lag/rolling NaN 제거 (최소 28일 히스토리 필요)
    lag_cols  = [c for c in df.columns if c.startswith("lag_")]
    roll_cols = [c for c in df.columns if c.startswith("roll_")]
    df = df.dropna(subset=lag_cols + roll_cols).reset_index(drop=True)

    # dtype 정리 & 카테고리 레벨 저장
    df = cast_dtypes(df)
    cat_levels = {c: list(df[c].cat.categories) for c in CAT_COLS if c in df.columns and str(df[c].dtype)=="category"}

    # 시간 기준 분할
    cutoff = df["date"].max() - pd.Timedelta(days=val_days)
    tr_mask = df["date"] <= cutoff
    va_mask = df["date"] >  cutoff

    feat_cols = get_feature_cols(df)
    cat_feats = [c for c in feat_cols if c in CAT_COLS and str(df[c].dtype)=="category"]

    X_tr, y_tr = df.loc[tr_mask, feat_cols], df.loc[tr_mask, TARGET]
    X_va, y_va = df.loc[va_mask, feat_cols], df.loc[va_mask, TARGET]

    params = LGBM_PARAMS.copy()
    params["objective"] = objective
    model = LGBMRegressor(**params)

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric=["rmse","mae"],
        categorical_feature=cat_feats,
        # early_stopping_rounds=200,
        # verbose=200
    )

    # 검증 메트릭
    pred_va = model.predict(X_va)
    rmse = mean_squared_error(y_va, pred_va)
    mae  = mean_absolute_error(y_va, pred_va)
    print(f"[VALID] RMSE {rmse:,.3f} | MAE {mae:,.3f} | SMAPE {smape(y_va.values, pred_va):.2f}%")

    return model, feat_cols, cat_levels


In [59]:
def _single_group_forecast_7days(
    hist: pd.DataFrame,
    model: LGBMRegressor,
    feat_cols: list[str],
    cat_levels: dict,
    clip_zero=True,
    round_pred=False
) -> pd.DataFrame:
    """
    hist: 단일 (venue,item) 그룹의 최근 '충분한' 히스토리 (적어도 28일)
          반드시 DATE_COL, TARGET 포함. 달력/lag/roll은 여기서 다시 계산.
    반환: 7일 예측 DataFrame [date, venue, item, yhat]
    """
    gcols = ["venue","item"]
    hist  = hist.sort_values("date").copy()
    if len(hist) < 28:
        raise ValueError("예측을 위해 그룹 히스토리에 최소 28일 데이터가 필요합니다.")

    # 상태(최근 타깃 28개) 초기화
    history_y = hist[TARGET].astype(float).tolist()
    last_date = hist["date"].max()
    rows = []

    # 그룹의 정적/준정적 피처(있으면) 최근값 유지
    last_row = hist.iloc[-1:].copy()

    for step in range(1, 8):  # 1..7일 예측
        cur_date = last_date + timedelta(days=step)

        # 새 로우 생성
        row = last_row.copy()
        row["date"] = cur_date
        row[DATE_COL] = cur_date

        # 달력 파생 재계산
        row = add_calendar_feats(row)

        # lag 1..28
        for lag in range(1, 29):
            row[f"lag_{lag}"] = history_y[-lag] if len(history_y) >= lag else np.nan

        # rolling (7,14,28) - 과거값만
        for win in [7,14,28]:
            if len(history_y) >= 2:
                vals = history_y[-win-1:-1] if len(history_y) >= win+1 else history_y[:-1]
                if len(vals) >= 2:
                    row[f"roll_mean_{win}"] = np.mean(vals)
                    row[f"roll_std_{win}"]  = np.std(vals, ddof=1) if len(vals) >= 2 else 0.0
                else:
                    row[f"roll_mean_{win}"] = np.nan
                    row[f"roll_std_{win}"]  = np.nan
            else:
                row[f"roll_mean_{win}"] = np.nan
                row[f"roll_std_{win}"]  = np.nan

        # # dtype & 카테고리 정합성
        # row = cast_dtypes(row, fit_categories=cat_levels)

        # # 예측
        # x = row[feat_cols].copy()
        # for c in feat_cols:          # 누락 피처는 0으로 보정
        #     if c not in x.columns:
        #         x[c] = 0

        # dtype 정리 및 카테고리 레벨 고정
        row = cast_dtypes(row, fit_categories=cat_levels)

        # === 누락 피처 안전 보강 ===
        for c in feat_cols:
            if c not in row.columns:
                if c in cat_levels:  # 카테고리 피처인 경우
                    row[c] = pd.Series(
                        [pd.NA],
                        dtype=CategoricalDtype(categories=cat_levels[c])
                    )
                else:                # 수치/불리언 피처인 경우
                    row[c] = 0

        # 이제 안전하게 선택
        x = row[feat_cols].copy()

        yhat = float(model.predict(x)[0])
        if clip_zero:
            yhat = max(0.0, yhat)
        if round_pred:
            yhat = float(np.rint(yhat))

        rows.append({
            "date":  cur_date,
            "venue": row["venue"].iloc[0],
            "item":  row["item"].iloc[0],
            "yhat":  yhat
        })

        # 방금 예측을 히스토리에 추가 -> 다음 스텝의 lag로 사용
        history_y.append(yhat)

    return pd.DataFrame(rows)

def predict_file_28to7(
    test_path: str,
    model: LGBMRegressor,
    feat_cols: list[str],
    cat_levels: dict,
    output_long_path: str | None = None,
    clip_zero=True,
    round_pred=False
) -> pd.DataFrame:
    """
    test_path: TEST_xx.csv (각 파일엔 각 (venue,item)의 최근 28일 이상 관측치가 들어있다고 가정)
    output_long_path: '영업일자 / 영업장명_메뉴명 / 매출수량'의 long 포맷 예측 결과 저장 경로(.csv)
    반환: long DF [영업일자, 영업장명_메뉴명, 매출수량]
    """
    test = pd.read_csv(test_path)
    test = ensure_venue_item(test)
    test[DATE_COL] = pd.to_datetime(test[DATE_COL])
    test = add_calendar_feats(test)
    test = test.sort_values(["venue","item","date"]).reset_index(drop=True)

    # 파일명에서 TEST 접두어 추출
    fname = os.path.basename(test_path)
    m = re.search(r"(TEST_\d+)", fname)
    test_prefix = m.group(1) if m else "TEST_XX"

    preds = []
    for (v,i), grp in test.groupby(["venue","item"], sort=False):
        grp = grp.copy()
        try:
            p = _single_group_forecast_7days(grp, model, feat_cols, cat_levels,
                                             clip_zero=clip_zero, round_pred=round_pred)
            # long 포맷으로 가공
            p["영업일자"] = [f"{test_prefix}+{k}일" for k in range(1, 8)]
            p["영업장명_메뉴명"] = f"{v}_{i}"
            p.rename(columns={"yhat": "매출수량"}, inplace=True)
            preds.append(p[["영업일자","영업장명_메뉴명","매출수량"]])
        except Exception as e:
            print(f"[WARN] ({v}, {i}) 예측 스킵: {e}")

    if len(preds) == 0:
        raise RuntimeError("예측할 그룹이 없습니다. TEST 파일의 내용(최근 28일 존재 여부)을 확인하세요.")

    pred_long = pd.concat(preds, ignore_index=True).sort_values(["영업일자","영업장명_메뉴명"]).reset_index(drop=True)

    if output_long_path:
        ensure_parent_dir(output_long_path)
        pred_long.to_csv(output_long_path, index=False, encoding="utf-8-sig")
        print(f"Saved long preds: {output_long_path} ({len(pred_long)} rows)")

    return pred_long


In [60]:
def align_and_cast_submission(
    pred_df: pd.DataFrame,
    sample_submission: pd.DataFrame,
    cast="int",          # "int" | "float32" | "float64"
    round_mode="round",  # "round" | "floor" | "ceil" | "none"
    decimals=0,
    clip_min=0.0,
    verbose=True
) -> pd.DataFrame:
    """
    pred_df: columns = [영업일자, 영업장명_메뉴명, 매출수량] (long 형식)
    sample_submission: 제출 포맷 기준 (영업일자 + 메뉴 컬럼들 wide 형식)

    반환: sample_submission과 동일한 순서/형태의 DataFrame
    """
    df = pred_df.copy()

    wide = (
        df.pivot_table(index="영업일자",
                       columns="영업장명_메뉴명",
                       values="매출수량",
                       aggfunc="sum")
          .astype(float)
    )

    # (선택) 행 정렬: TEST_xx + n일
    idx_series = wide.index.to_series()
    key = idx_series.str.extract(r"TEST_(\d+)\+(\d+)일")
    if key.notna().all().all():
        key = key.astype(int)
        order = np.lexsort((key[1].values, key[0].values))  # (day, test_no)
        wide = wide.iloc[order]

    final = sample_submission.copy().set_index("영업일자")

    missing_cols = [c for c in final.columns if c not in wide.columns]
    extra_cols   = [c for c in wide.columns  if c not in final.columns]
    if verbose:
        if missing_cols:
            print(f"[WARN] wide에 없는 열({len(missing_cols)}):", missing_cols[:5], "...")
        if extra_cols:
            print(f"[WARN] sample_submission에 없는 예측 열({len(extra_cols)}):", extra_cols[:5], "...")

    aligned = (
        wide.reindex(index=final.index, fill_value=0.0)
            .reindex(columns=final.columns, fill_value=0.0)
    )

    vals = aligned.values
    if clip_min is not None:
        vals = np.clip(vals, clip_min, None)

    if round_mode != "none":
        if round_mode == "round":
            vals = np.round(vals, decimals=decimals)
        elif round_mode == "floor":
            vals = np.floor(vals)
        elif round_mode == "ceil":
            vals = np.ceil(vals)
        else:
            raise ValueError("round_mode must be one of: round|floor|ceil|none")

    if cast == "int":
        vals = np.rint(vals).astype(np.int64)
    elif cast == "float32":
        vals = vals.astype(np.float32)
    elif cast == "float64":
        vals = vals.astype(np.float64)
    else:
        raise ValueError("cast must be one of: int|float32|float64")

    final.loc[:, :] = vals
    final = final.reset_index()
    return final


In [ ]:
# 1) 학습 데이터 로드
train_path = os.path.join(DATA_DIR, "train.csv")
if "train" in globals():
    train_df = train.copy()
elif os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    raise FileNotFoundError("train DataFrame이 메모리에 없고, ../data/train.csv 파일도 없습니다.")

# 2) 학습
model, feat_cols, cat_levels = train_lgbm_28to7(train_df, val_days=VAL_DAYS, objective=LGBM_OBJECTIVE)

# 3) 모델 저장
model_path = os.path.join(MODEL_DIR, "lgbm_28to7.pkl")
ensure_parent_dir(model_path)
joblib.dump({"model": model, "feat_cols": feat_cols, "cat_levels": cat_levels}, model_path)
print(f"Model saved: {model_path}")

# 4) TEST_00 ~ TEST_09 예측 (long 포맷 누적)
all_preds = []
test_files = sorted(glob.glob(os.path.join(TEST_DIR, "TEST_*.csv")))
if not test_files:
    print(f"[INFO] 테스트 파일이 없습니다: {TEST_DIR}/TEST_*.csv")
for path in test_files:
    try:
        pred_long = predict_file_28to7(
            test_path=path,
            model=model,
            feat_cols=feat_cols,
            cat_levels=cat_levels,
            output_long_path=None,        # 파일로도 저장하려면 경로 지정
            clip_zero=CLIP_ZERO,
            round_pred=ROUND_PRED
        )
        all_preds.append(pred_long)
        print(f"Predicted {os.path.basename(path)} -> {len(pred_long)} rows")
    except Exception as e:
        print(f"[ERROR] {os.path.basename(path)} 예측 실패: {e}")

if not all_preds:
    raise RuntimeError("어떤 테스트 파일에서도 예측을 생성하지 못했습니다.")
full_pred_df = pd.concat(all_preds, ignore_index=True)

# 5) 제출 포맷 정렬/캐스팅
sample_path = os.path.join(DATA_DIR, "sample_submission.csv")
if not os.path.exists(sample_path):
    raise FileNotFoundError(f"sample_submission.csv가 없습니다: {sample_path}")

sample_submission = pd.read_csv(sample_path)

submission = align_and_cast_submission(
    pred_df=full_pred_df,
    sample_submission=sample_submission,
    cast="int",           # 제출이 정수면 int 권장
    round_mode="round",   # 반올림
    decimals=0,
    clip_min=0.0,
    verbose=True
)

# 6) 저장
sub_path = os.path.join(OUT_DIR, "submission.csv")
ensure_parent_dir(sub_path)
submission.to_csv(sub_path, index=False, encoding="utf-8-sig")
print(f"Saved submission -> {sub_path}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014927 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8922
[LightGBM] [Info] Number of data points in the train set: 90517, number of used features: 52
[LightGBM] [Info] Start training from score 2.331343
[VALID] RMSE 212.798 | MAE 4.441 | SMAPE 132.92%
Model saved: ../model\lgbm_28to7.pkl
Predicted TEST_00.csv -> 1351 rows
Predicted TEST_01.csv -> 1351 rows
Predicted TEST_02.csv -> 1351 rows
Predicted TEST_03.csv -> 1351 rows
Predicted TEST_04.csv -> 1351 rows
Predicted TEST_05.csv -> 1351 rows
Predicted TEST_06.csv -> 1351 rows
Predicted TEST_07.csv -> 1351 rows
Predicted TEST_08.csv -> 1351 rows
Predicted TEST_09.csv -> 1351 rows
Saved submission -> ../output\baseline_submission.csv


## CatBoost

## GRU

## Validation